In [1]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone

In [2]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
loader = TextLoader("/data")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

RuntimeError: Error loading ../../modules/state_of_the_union.txt